In [1]:
import pandas as pd

In [10]:
df = pd.read_csv(r'C:\Simulation\data\10_mod\3_KC_case_data.csv')

- date – дата совершения события
- event - событие  
•	app_install – установка приложения  
•	app_start – открыть приложения  
•	registration – зарегистрироваться   
•	search – перейти на страницу поиска товаров (каталог)  
•	open_item – открыть товар  
•	choose_item – отправить товар в корзину  
•	tap_basket – перейти в корзину  
•	purchase – подтверждение покупки  
- purchase_sum – стоимость покупки (при совершении события ‘purchase’)
- os_name – платформа пользователя
- device_id – идентификатор устройства пользователя
- gender – пол пользователя
- city – город пользователя
- utm_source – канал, с которого пришел пользователь
•	yandex-direct – Яндекс директ  
•	google_ads – реклама в Google  
•	vk_ads – реклама в ВК  
•	instagram_ads – реклама в instagram  
•	facebook_ads – реклама в facebook  
•	referral – акция «приведи друга»  
•	Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы    


In [11]:
df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747968 entries, 0 to 2747967
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   date          object 
 1   event         object 
 2   purchase_sum  float64
 3   os_name       object 
 4   device_id     int64  
 5   gender        object 
 6   city          object 
 7   utm_source    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 167.7+ MB


In [13]:
df.describe()

,purchase_sum,device_id
count,141383.000000,2.747968e+06
mean,709.116092,1.090841e+07
std,1289.458186,9.871445e+06
min,61.000000,4.013000e+03
25%,226.500000,1.975336e+06
50%,394.000000,8.731416e+06
75%,560.500000,1.736726e+07
max,7884.000000,3.539777e+07


In [14]:
df.isna().sum()

date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

In [15]:
df['date'] = pd.to_datetime(df['date'])

In [16]:
df['event'].value_counts()

event
app_start      748705
search         708639
choose_item    538669
tap_basket     377665
app_install    154597
purchase       141383
register        78310
Name: count, dtype: int64

MAU февраля

In [17]:
df[df['date'].dt.month == 2]['device_id'].nunique()

75032

Количество установок в январе

In [18]:
df.query('date.dt.month == 1 and event == "app_install"').event.count()

np.int64(80297)

In [19]:
df[(df['date'].dt.month == 1) & (df['event'] == 'app_install')].event.count()

np.int64(80297)

Присвойте пользователям когорты по дню установки приложения и посчитайте для них конверсию из установки в покупку в течении 7 дней. Для какой когорты конверсия была наиболее наибольшей? Ответ в формате дд.мм.гггг. Пользователь считается сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней

In [20]:
# вариант из ответов, предназначенный для дальнейшей выгрузки в csv и обработке в Tablo
installs = df[df['event'] == 'app_install'][['date', 'device_id']].drop_duplicates('device_id').rename(columns={'date': 'install_date'})
purchase = df[df['event'] == 'purchase'][['date', 'device_id']].drop_duplicates('device_id').rename(columns={'date': 'first_purchase_date'})
cr = df.merge(installs, on='device_id').merge(purchase, how='left', on='device_id')
cr

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date,first_purchase_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01
2,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,NaT
3,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,NaT
4,2020-01-01,app_start,NaN,android,5148476,male,Saint-Petersburg,-,2020-01-01,2020-01-24
...,...,...,...,...,...,...,...,...,...,...
2263018,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03-28,2020-03-31
2263019,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03-31,2020-03-31
2263020,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03-31,2020-03-31
2263021,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03-06,2020-03-31


In [28]:
installs = (df[df['event'] == 'app_install'].sort_values('date').drop_duplicates('device_id').assign(install_date=lambda x: x['date'].dt.floor('D'))[['device_id', 'install_date']])

In [22]:
purchases = (df[df['event'] == 'purchase'].sort_values('date').drop_duplicates('device_id').rename(columns={'date': 'purchase_date'})[['device_id', 'purchase_date']])

In [23]:
cohort = installs.merge(purchases, on='device_id', how='left')

In [24]:
cohort['purchase_7d'] = (cohort['purchase_date']<= cohort['install_date'] + pd.Timedelta(days=7))

In [25]:
cohort

,device_id,install_date,purchase_date,purchase_7d
0,4921563,2020-01-01,2020-01-10,False
1,33180892,2020-01-01,NaT,False
2,3331097,2020-01-01,2020-01-02,True
3,1741756,2020-01-01,2020-01-19,False
4,5420901,2020-01-01,2020-01-04,True
...,...,...,...,...
154592,1757461,2020-03-31,2020-03-31,True
154593,32851278,2020-03-31,NaT,False
154594,8215181,2020-03-31,NaT,False
154595,15401367,2020-03-31,NaT,False


In [45]:
cohort.groupby('install_date').agg(installs=('device_id', 'count'), buyers_7d=('purchase_7d', 'sum')).assign(cr_7d=lambda x: x['buyers_7d'] / x['installs']).round(3).assign(cr_7d_per=lambda x: x['cr_7d']*100).round(3)

,installs,buyers_7d,cr_7d,cr_7d_per
install_date,,,,
2020-01-01,3579,1408,0.393,39.3
2020-01-02,3144,1186,0.377,37.7
2020-01-03,2402,834,0.347,34.7
2020-01-04,1831,639,0.349,34.9
2020-01-05,1671,587,0.351,35.1
...,...,...,...,...
2020-03-27,1199,277,0.231,23.1
2020-03-28,1091,209,0.192,19.2
2020-03-29,1117,223,0.200,20.0


С какого платного маркетингового канала пришло больше всего пользователей?

In [57]:
df[(df['utm_source'] != '-') & (df['event'] == 'app_install')].drop_duplicates('device_id').groupby('utm_source').agg({'device_id': 'count'}).sort_values('device_id', ascending=False)

,device_id
utm_source,
yandex-direct,29368
google_ads,26286
vk_ads,23189
instagram_ads,20096
facebook_ads,13916
referal,9282


In [56]:
df[(df['utm_source'] != '-') & (df['event'] == 'app_install')].drop_duplicates('device_id').groupby('utm_source').agg(count_user=('device_id', 'count')).sort_values('count_user', ascending=False)

,count_user
utm_source,
yandex-direct,29368
google_ads,26286
vk_ads,23189
instagram_ads,20096
facebook_ads,13916
referal,9282


Проанализируйте на каком этапе воронки отваливается большая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?  
В ответ впишите один из шагов, конверсия из которого (в следующий шаг) самая низкая, в таком формате:  
«Поиск», «Добавление товара», «Переход в корзину», «Регистрация».  
Подсказка: Для того, чтобы выделить группу тех, кому нужно и не нужно регистрироваться, добавьте колонку с датой регистрации. Если дата регистрации < даты совершения события, то пользователь уже зарегистрирован.

In [64]:
installs = (df[df['event'] == 'app_install'].sort_values('date').drop_duplicates('device_id').assign(install_date=lambda x: x['date'].dt.floor('D'))[['device_id', 'install_date']])

In [65]:
app_start = (df[df['event'] == 'app_start'].sort_values('date').drop_duplicates('device_id').rename(columns={'date': 'app_start_date'})[['device_id', 'app_start_date']])

In [66]:
registration = (df[df['event'] == 'registration'].sort_values('date').drop_duplicates('device_id').rename(columns={'date': 'registration_date'})[['device_id', 'registration_date']])

In [67]:
search = (df[df['event'] == 'search'].sort_values('date').drop_duplicates('device_id').rename(columns={'date': 'search_date'})[['device_id', 'search_date']])

In [68]:
open_item = (df[df['event'] == 'open_item'].sort_values('date').drop_duplicates('device_id').rename(columns={'date': 'open_item'})[['device_id', 'open_item']])

In [69]:
choose_item = (df[df['event'] == 'choose_item'].sort_values('date').drop_duplicates('device_id').rename(columns={'date': 'choose_item'})[['device_id', 'choose_item']])

In [70]:
tap_basket = (df[df['event'] == 'tap_basket'].sort_values('date').drop_duplicates('device_id').rename(columns={'date': 'tap_basket'})[['device_id', 'tap_basket']])

In [71]:
purchases = (df[df['event'] == 'purchase'].sort_values('date').drop_duplicates('device_id').rename(columns={'date': 'purchase_date'})[['device_id', 'purchase_date']])

In [72]:
funnel = (
    installs
    .merge(app_start, on='device_id', how='left')
    .merge(registration, on='device_id', how='left')
    .merge(search, on='device_id', how='left')
    .merge(open_item, on='device_id', how='left')
    .merge(choose_item, on='device_id', how='left')
    .merge(tap_basket, on='device_id', how='left')
    .merge(purchases, on='device_id', how='left')
)
funnel

,device_id,install_date,app_start_date,registration_date,search_date,open_item,choose_item,tap_basket,purchase_date
0,4921563,2020-01-01,2020-01-01,NaT,2020-01-01,NaT,2020-01-01,2020-01-01,2020-01-10
1,33180892,2020-01-01,2020-01-01,NaT,2020-01-01,NaT,NaT,NaT,NaT
2,3331097,2020-01-01,2020-01-01,NaT,2020-01-01,NaT,2020-01-02,2020-01-02,2020-01-02
3,1741756,2020-01-01,2020-01-01,NaT,2020-01-01,NaT,2020-01-01,2020-01-01,2020-01-19
4,5420901,2020-01-01,2020-01-01,NaT,2020-01-01,NaT,2020-01-01,2020-01-01,2020-01-04
...,...,...,...,...,...,...,...,...,...
154592,1757461,2020-03-31,2020-03-31,NaT,2020-03-31,NaT,2020-03-31,2020-03-31,2020-03-31
154593,32851278,2020-03-31,2020-03-31,NaT,2020-03-31,NaT,NaT,NaT,NaT
154594,8215181,2020-03-31,2020-03-31,NaT,2020-03-31,NaT,2020-03-31,2020-03-31,NaT
154595,15401367,2020-03-31,2020-03-31,NaT,2020-03-31,NaT,2020-03-31,NaT,NaT
